In [12]:
# NeuMF training + leave-one-out evaluation (sample 99 negatives per positive)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import random
import os

# -------------------------
# Paths
# -------------------------
train_path = './output_neumf/neumf_train.csv'
test_path  = './output_neumf/neumf_test.csv'

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

# Keep only necessary cols and ensure types
train_df = train_df[['user','item','label']].copy()
test_df  = test_df[['user','item','label']].copy()

# Ensure positive-only test for holdout evaluation
test_pos_df = test_df[test_df['label'] == 1].copy()
if test_pos_df.empty:
    raise RuntimeError("Không tìm thấy sample positive trong test_df (label==1). Hãy đảm bảo test chứa positives holdout.")

# -------------------------
# Encode users/items
# -------------------------
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# Fit encoders on union of train+test to avoid unseen labels at inference
all_users = pd.concat([train_df['user'], test_pos_df['user']]).astype(str)
all_items = pd.concat([train_df['item'], test_pos_df['item']]).astype(str)
user_encoder.fit(all_users)
item_encoder.fit(all_items)

train_df['user'] = user_encoder.transform(train_df['user'].astype(str))
train_df['item'] = item_encoder.transform(train_df['item'].astype(str))
test_pos_df['user'] = user_encoder.transform(test_pos_df['user'].astype(str))
test_pos_df['item'] = item_encoder.transform(test_pos_df['item'].astype(str))

n_users = len(user_encoder.classes_)
n_items = len(item_encoder.classes_)

print(f"Users: {n_users}, Items: {n_items}, Train rows: {len(train_df)}, Test positives: {len(test_pos_df)}")

# -------------------------
# Build train interaction set (to avoid sampling seen items as negatives)
# -------------------------
train_user_pos = train_df[train_df['label']==1].groupby('user')['item'].apply(set).to_dict()
# ensure users with no positives have empty set
for u in range(n_users):
    if u not in train_user_pos:
        train_user_pos[u] = set()

# -------------------------
# Dataset & DataLoader (pointwise)
# -------------------------
class RatingDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user'].values, dtype=torch.long)
        self.items = torch.tensor(df['item'].values, dtype=torch.long)
        self.labels = torch.tensor(df['label'].values, dtype=torch.float32)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

train_loader = DataLoader(RatingDataset(train_df), batch_size=1024, shuffle=True, num_workers=0)

# -------------------------
# NeuMF model (slightly larger)
# -------------------------
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, mf_dim=32, mlp_layers=[128,64,32,16], dropout=0.0):
        super().__init__()
        self.user_emb_gmf = nn.Embedding(n_users, mf_dim)
        self.item_emb_gmf = nn.Embedding(n_items, mf_dim)
        self.user_emb_mlp = nn.Embedding(n_users, mlp_layers[0]//2)
        self.item_emb_mlp = nn.Embedding(n_items, mlp_layers[0]//2)
        mlp = []
        input_dim = mlp_layers[0]
        for dim in mlp_layers[1:]:
            mlp.append(nn.Linear(input_dim, dim))
            mlp.append(nn.ReLU())
            if dropout>0:
                mlp.append(nn.Dropout(dropout))
            input_dim = dim
        self.mlp = nn.Sequential(*mlp)
        self.fc = nn.Linear(mf_dim + mlp_layers[-1], 1)
        self.sigmoid = nn.Sigmoid()
        # init
        nn.init.normal_(self.user_emb_gmf.weight, std=0.01)
        nn.init.normal_(self.item_emb_gmf.weight, std=0.01)
        nn.init.normal_(self.user_emb_mlp.weight, std=0.01)
        nn.init.normal_(self.item_emb_mlp.weight, std=0.01)

    def forward(self, user, item):
        gmf = self.user_emb_gmf(user) * self.item_emb_gmf(item)
        mlp_in = torch.cat([self.user_emb_mlp(user), self.item_emb_mlp(item)], dim=-1)
        mlp_out = self.mlp(mlp_in)
        concat = torch.cat([gmf, mlp_out], dim=-1)
        out = self.fc(concat)
        return self.sigmoid(out).squeeze()

# -------------------------
# Training setup
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuMF(n_users, n_items, mf_dim=32, mlp_layers=[128,64,32,16], dropout=0.0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.BCELoss()

# -------------------------
# Train
# -------------------------
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for users, items, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        users = users.to(device); items = items.to(device); labels = labels.to(device)
        preds = model(users, items)
        loss = criterion(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * users.size(0)
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: avg_loss={avg_loss:.6f}")

# -------------------------
# Evaluation: leave-one-out with sampled negatives
# For each positive (user, pos_item) in test_pos_df:
#   sample N_NEG negatives from items NOT in user's train positives AND NOT equal pos_item
#   rank pos_item among negatives by predicted score
# -------------------------
def precision_at_k_single(pos_index, ranked_list, k=10):
    return 1.0 if pos_index < k else 0.0

def average_precision_single(pos_index, ranked_list, k=10):
    # In leave-one-out with single positive, AP is 1/(rank) if rank<=k else 0
    if pos_index < k:
        return 1.0 / (pos_index + 1.0)
    return 0.0

def evaluate_leave_one_out(model, test_pos_df, train_user_pos, n_items, n_negatives=99, k=10, seed=42):
    rng = random.Random(seed)
    model.eval()
    precisions = []
    aps = []
    users = test_pos_df['user'].values
    items = test_pos_df['item'].values
    with torch.no_grad():
        for u, pos_item in tqdm(zip(users, items), total=len(users), desc="Eval LOO"):
            # sample negatives
            neg_pool = set(range(n_items)) - train_user_pos.get(u, set()) - {pos_item}
            # if pool smaller than required, take all
            neg_list = rng.sample(list(neg_pool), k=min(n_negatives, len(neg_pool)))
            candidate_items = neg_list + [pos_item]
            # compute scores
            user_tensor = torch.tensor([u]*len(candidate_items), dtype=torch.long).to(device)
            item_tensor = torch.tensor(candidate_items, dtype=torch.long).to(device)
            scores = model(user_tensor, item_tensor).cpu().numpy()
            # rank descending
            ranked_idx = np.argsort(-scores)
            # find rank (0-based) of pos_item
            pos_rank = int(np.where(np.array(candidate_items)[ranked_idx] == pos_item)[0][0])
            precisions.append(precision_at_k_single(pos_rank, ranked_idx, k))
            aps.append(average_precision_single(pos_rank, ranked_idx, k))
    return np.mean(precisions), np.mean(aps)

precision10, map10 = evaluate_leave_one_out(model, test_pos_df, train_user_pos, n_items, n_negatives=99, k=10)
print("===================================================")
print(f"Precision@10: {precision10:.4f}")
print(f"MAP@10:       {map10:.4f}")
print("===================================================")


Users: 42, Items: 139, Train rows: 3373, Test positives: 217


Epoch 1/10: 100%|██████████| 4/4 [00:00<00:00, 91.53it/s]


Epoch 1: avg_loss=0.714706


Epoch 2/10: 100%|██████████| 4/4 [00:00<00:00, 82.33it/s]


Epoch 2: avg_loss=0.709957


...

Epoch 9: avg_loss=0.666274


Epoch 10/10: 100%|██████████| 4/4 [00:00<00:00, 83.53it/s]


Epoch 10: avg_loss=0.650796


Eval LOO: 100%|██████████| 217/217 [00:00<00:00, 3189.63it/s]

Precision@10: 0.3548
MAP@10:       0.1263


In [13]:
# =====================================================
# LNCM (Latent Neural Context Model)
# =====================================================
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# =====================================================
# 1️⃣ LOAD & CHUẨN BỊ DỮ LIỆU
# =====================================================
# Đọc file dữ liệu gồm user, item, label (rating) và context_id
df = pd.read_csv('output_carskit_clean/ratings_with_context_id.csv')

# 🔸 Mục đích: ánh xạ user/item/context sang ID liên tục (0,1,2,...)
#  -> giúp embedding layer có thể xử lý được
user2id = {u: i for i, u in enumerate(df['user'].unique())}
item2id = {i: j for j, i in enumerate(df['item'].unique())}
context2id = {c: k for k, c in enumerate(df['context_id'].unique())}

# Thay thế các giá trị gốc bằng ID
df['user'] = df['user'].map(user2id)
df['item'] = df['item'].map(item2id)
df['context'] = df['context_id'].map(context2id)

# Thống kê số lượng phần tử
n_users = len(user2id)
n_items = len(item2id)
n_contexts = len(context2id)
print(f"Users: {n_users}, Items: {n_items}, Contexts: {n_contexts}")

# 🔹 Chia dữ liệu thành tập huấn luyện và kiểm thử
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# =====================================================
# 1.1️⃣ ĐỊNH NGHĨA Dataset CHO PYTORCH
# =====================================================
class RatingDataset(torch.utils.data.Dataset):
    """Dataset cơ bản cho bài toán gợi ý có ngữ cảnh"""
    def __init__(self, df):
        # Ép kiểu về tensor để tiện cho GPU
        self.users = torch.tensor(df['user'].values, dtype=torch.long)
        self.items = torch.tensor(df['item'].values, dtype=torch.long)
        self.contexts = torch.tensor(df['context'].values, dtype=torch.long)
        self.labels = torch.tensor(df['label'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.contexts[idx], self.labels[idx]

# Tạo DataLoader để huấn luyện theo batch
train_data = RatingDataset(train_df)
test_data = RatingDataset(test_df)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=False)

# =====================================================
# 2️⃣ MÔ HÌNH: LNCM (Latent Neural Context Model)
# =====================================================
class LNCM_Fixed(nn.Module):
    """
    🔹 Ý tưởng:
        - LNCM không dùng embedding context trực tiếp.
        - Thay vào đó, mô hình "tự sinh" vector ngữ cảnh tiềm ẩn (latent context)
          từ cặp (user, item) thông qua một encoder kiểu VAE (Variational Autoencoder).
    🔹 Mục tiêu:
        - Học được tác động ngầm của ngữ cảnh ngay cả khi context không rõ ràng.
    """

    def __init__(self, n_users, n_items, n_contexts, emb_dim=32, latent_dim=16):
        super().__init__()

        # 1️⃣ Embedding cho user và item
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)

        # 2️⃣ Encoder sinh ngữ cảnh tiềm ẩn (latent context)
        #    Input: [user_emb, item_emb]
        #    Output: [mu, logvar] dùng cho VAE reparameterization trick
        self.context_encoder = nn.Sequential(
            nn.Linear(emb_dim * 2, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, emb_dim * 2)  # output gồm 2 phần: mu & logvar
        )

        # 3️⃣ Mạng fully-connected dự đoán xác suất user thích item
        self.fc = nn.Sequential(
            nn.Linear(emb_dim * 3, 64),  # user + item + latent context
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        # 4️⃣ Khởi tạo trọng số embedding
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)

    # === Hàm tái tham số hóa trong VAE ===
    def reparameterize(self, mu, logvar):
        """Chuyển (mu, logvar) → latent vector theo công thức: z = mu + eps * sigma"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, user, item, context=None):
        """Lan truyền tiến (forward pass)"""
        # Embedding user và item
        u = self.user_emb(user)
        i = self.item_emb(item)

        # Encoder sinh (mu, logvar) → latent context
        context_params = self.context_encoder(torch.cat([u, i], dim=1))
        mu, logvar = torch.chunk(context_params, 2, dim=1)
        c_latent = self.reparameterize(mu, logvar)

        # Ghép tất cả embedding lại để dự đoán
        x = torch.cat([u, i, c_latent], dim=1)
        return self.fc(x).squeeze()

# =====================================================
# 3️⃣ VÒNG HUẤN LUYỆN MÔ HÌNH
# =====================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_lncm = LNCM_Fixed(n_users, n_items, n_contexts).to(device)
optimizer = optim.Adam(model_lncm.parameters(), lr=0.001)
loss_fn = nn.BCELoss()  # Binary Cross-Entropy cho nhãn 0/1

print("\n=== Training LNCM_Fixed ===")
for epoch in range(10):
    model_lncm.train()
    total_loss = 0

    # Duyệt từng batch dữ liệu
    for users, items, contexts, labels in tqdm(train_loader, desc=f"[LNCM] Epoch {epoch+1}"):
        users, items, labels = users.to(device), items.to(device), labels.to(device)

        preds = model_lncm(users, items)      # Dự đoán
        loss = loss_fn(preds, labels)         # Tính loss

        optimizer.zero_grad()
        loss.backward()                       # Lan truyền ngược
        optimizer.step()                       # Cập nhật trọng số

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Train Loss: {total_loss / len(train_loader):.4f}")

# =====================================================
# 4️⃣ ĐÁNH GIÁ MÔ HÌNH
# =====================================================
def evaluate_ranking_metrics(model, test_loader, device, K=10):
    """
    Đánh giá mô hình theo:
    - Basic metrics: AUC, RMSE, MAE
    - Ranking metrics: Precision@10, Recall@10, Hit@10, MAP@10
    """
    model.eval()
    user_preds = {}
    all_preds, all_labels = [], []

    with torch.no_grad():
        for users, items, contexts, labels in test_loader:
            users, items, contexts = users.to(device), items.to(device), contexts.to(device)
            preds = model(users, items, contexts).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

            for u, i, p, l in zip(users.cpu().numpy(), items.cpu().numpy(), preds, labels.numpy()):
                if u not in user_preds:
                    user_preds[u] = []
                user_preds[u].append((i, p, l))

    # === Basic metrics ===
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    auc = roc_auc_score(all_labels, all_preds)
    rmse = np.sqrt(np.mean((all_labels - all_preds) ** 2))
    mae = np.mean(np.abs(all_labels - all_preds))

    print(f"\n--- Basic Metrics ---")
    print(f"AUC: {auc:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")

    # === Ranking metrics @K ===
    precisions, recalls, hits, maps = [], [], [], []

    for user, predictions in user_preds.items():
        predictions.sort(key=lambda x: x[1], reverse=True)
        total_relevant = sum([l for _, _, l in predictions])
        if total_relevant == 0:
            continue

        top_k = predictions[:K]
        top_k_labels = [l for _, _, l in top_k]

        # Precision, Recall, Hit
        precision = sum(top_k_labels) / K
        recall = sum(top_k_labels) / total_relevant
        hit = 1.0 if sum(top_k_labels) > 0 else 0.0

        # MAP@K
        ap_sum, correct = 0.0, 0
        for idx, (_, _, label) in enumerate(top_k, start=1):
            if label == 1:
                correct += 1
                ap_sum += correct / idx
        map_k = ap_sum / min(total_relevant, K)

        precisions.append(precision)
        recalls.append(recall)
        hits.append(hit)
        maps.append(map_k)

    # In kết quả trung bình
    print(f"\n--- Ranking Metrics @ {K} ---")
    print(f"Precision@{K}: {np.mean(precisions):.4f}")
    print(f"Recall@{K}:    {np.mean(recalls):.4f}")
    print(f"Hit@{K}:       {np.mean(hits):.4f}")
    print(f"MAP@{K}:       {np.mean(maps):.4f}")


# =====================================================
# 5️⃣ CHẠY ĐÁNH GIÁ SAU HUẤN LUYỆN
# =====================================================
print("\n=== Evaluating  ===")
evaluate_ranking_metrics(model_lncm, test_loader, device)

print("\n✅ LNCM Training & Evaluation Complete!")


Users: 42, Items: 139, Contexts: 27
Using device: cpu

=== Training LNCM_Fixed ===


[LNCM] Epoch 1: 100%|██████████| 51/51 [00:00<00:00, 390.36it/s]


Epoch 1 | Train Loss: 0.6332


[LNCM] Epoch 2: 100%|██████████| 51/51 [00:00<00:00, 357.57it/s]


Epoch 2 | Train Loss: 0.5569


...

Epoch 8 | Train Loss: 0.4252


[LNCM] Epoch 9: 100%|██████████| 51/51 [00:00<00:00, 341.40it/s]


Epoch 9 | Train Loss: 0.4247


[LNCM] Epoch 10: 100%|██████████| 51/51 [00:00<00:00, 350.93it/s]

Epoch 10 | Train Loss: 0.4158

=== Evaluating  ===

--- Basic Metrics ---
AUC: 0.7841
RMSE: 0.3911
MAE: 0.2851

--- Ranking Metrics @ 10 ---
Precision@10: 0.3414
Recall@10:    0.7727
Hit@10:       1.0000
MAP@10:       0.5073

✅ LNCM Training & Evaluation Complete!


In [14]:
# =====================================================
# 📚 Explicit Neural Context Model (ENCM)
# =====================================================

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import numpy as np

# -----------------------------------------------------
# TẢI VÀ XỬ LÝ DỮ LIỆU
# -----------------------------------------------------
# Dữ liệu gồm 4 cột chính: user, item, label (0/1), context_id
# context_id thể hiện các điều kiện như thời tiết, tâm trạng, đường, v.v.
df = pd.read_csv('output_carskit_clean/ratings_with_context_id.csv')

# Ánh xạ id rời rạc thành chỉ số liên tục để dùng cho embedding
user2id = {u: i for i, u in enumerate(df['user'].unique())}
item2id = {i: j for j, i in enumerate(df['item'].unique())}
context2id = {c: k for k, c in enumerate(df['context_id'].unique())}

# Gán lại id liên tục
df['user'] = df['user'].map(user2id)
df['item'] = df['item'].map(item2id)
df['context'] = df['context_id'].map(context2id)

# Thống kê số lượng thực thể
n_users = len(user2id)
n_items = len(item2id)
n_contexts = len(context2id)
print(f"Users: {n_users}, Items: {n_items}, Contexts: {n_contexts}")

# Chia train/test (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# -----------------------------------------------------
# TẠO DATASET CHO PYTORCH
# -----------------------------------------------------
class RatingDataset(torch.utils.data.Dataset):
    """Dataset chuẩn cho bài toán recommendation."""
    def __init__(self, df):
        self.users = torch.tensor(df['user'].values, dtype=torch.long)
        self.items = torch.tensor(df['item'].values, dtype=torch.long)
        self.contexts = torch.tensor(df['context'].values, dtype=torch.long)
        self.labels = torch.tensor(df['label'].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.contexts[idx], self.labels[idx]

# DataLoader để chia batch
train_data = RatingDataset(train_df)
test_data = RatingDataset(test_df)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=False)

# -----------------------------------------------------
# ĐỊNH NGHĨA MÔ HÌNH ENCM
# -----------------------------------------------------
class ENCM(nn.Module):
    """
    ENCM - Explicit Neural Context Model
    ------------------------------------
    Ý tưởng:
      - Mỗi user, item, context được biểu diễn bằng embedding.
      - Context không chỉ được nối vào mà còn *điều chỉnh (modulate)*
        embedding của user và item thông qua *gating mechanism*.
      - Mục tiêu: học ảnh hưởng của context đến hành vi user rõ ràng hơn.

    Kiến trúc:
      user/item embedding → context gate → modulation → concat → MLP → output
    """
    def __init__(self, n_users, n_items, n_contexts, emb_dim=32):
        super().__init__()
        
        # --- 3 loại embedding cơ bản ---
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.context_emb = nn.Embedding(n_contexts, emb_dim)
        
        # --- Gating: context ảnh hưởng lên user/item ---
        # Gate là 1 hàm sigmoid để tạo trọng số điều chỉnh động cho từng chiều embedding
        self.user_gate = nn.Sequential(
            nn.Linear(emb_dim * 2, emb_dim),  # [user, context] -> gate vector
            nn.Sigmoid()
        )
        self.item_gate = nn.Sequential(
            nn.Linear(emb_dim * 2, emb_dim),  # [item, context] -> gate vector
            nn.Sigmoid()
        )
        
        # --- Mạng dự đoán (prediction network) ---
        # Nhận đầu vào là [user_modulated, item_modulated, context]
        self.fc = nn.Sequential(
            nn.Linear(emb_dim * 3, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # đầu ra xác suất
        )
        
        # --- Khởi tạo trọng số embedding ---
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        nn.init.xavier_uniform_(self.context_emb.weight)
    
    def forward(self, user, item, context):
        # 1️⃣ Lấy embedding cơ bản
        u = self.user_emb(user)
        i = self.item_emb(item)
        c = self.context_emb(context)
        
        # 2️⃣ Context điều chỉnh user
        u_context = torch.cat([u, c], dim=1)
        u_gate = self.user_gate(u_context)      # gate ∈ [0,1]^emb_dim
        u_modulated = u * u_gate                # nhân từng chiều để điều chỉnh
        
        # 3️⃣ Context điều chỉnh item
        i_context = torch.cat([i, c], dim=1)
        i_gate = self.item_gate(i_context)
        i_modulated = i * i_gate
        
        # 4️⃣ Kết hợp tất cả embeddings lại
        x = torch.cat([u_modulated, i_modulated, c], dim=1)
        
        # 5️⃣ Dự đoán xác suất user thích item trong context
        return self.fc(x).squeeze()

# -----------------------------------------------------
# HUẤN LUYỆN MÔ HÌNH
# -----------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = ENCM(n_users, n_items, n_contexts, emb_dim=32).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = nn.BCELoss()  # Vì bài toán nhị phân (0/1)

print("\n=== Training ENCM ===")
for epoch in range(10):
    model.train()
    total_loss = 0
    
    # Duyệt từng batch trong dữ liệu huấn luyện
    for users, items, contexts, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        users, items, contexts, labels = users.to(device), items.to(device), contexts.to(device), labels.to(device)
        
        preds = model(users, items, contexts)          # dự đoán
        loss = loss_fn(preds, labels)                  # tính lỗi
        optimizer.zero_grad()
        loss.backward()                                # lan truyền ngược
        optimizer.step()                               # cập nhật trọng số
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1} | Train Loss: {total_loss / len(train_loader):.4f}")

# -----------------------------------------------------
#  ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH
# -----------------------------------------------------
print("\n=== Evaluating ENCM ===")

# -----------------------------------------------------
# ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH (CHỈ TẠI K=10)
# -----------------------------------------------------
print("\n=== Evaluating ENCM ===")

def evaluate_ranking_metrics(model, test_loader, device, K=10):
    """
    Đánh giá mô hình theo:
    - Basic metrics: AUC, RMSE, MAE
    - Ranking metrics: Precision@10, Recall@10, Hit@10, MAP@10
    """
    model.eval()
    user_preds = {}
    all_preds, all_labels = [], []

    with torch.no_grad():
        for users, items, contexts, labels in test_loader:
            users, items, contexts = users.to(device), items.to(device), contexts.to(device)
            preds = model(users, items, contexts).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

            for u, i, p, l in zip(users.cpu().numpy(), items.cpu().numpy(), preds, labels.numpy()):
                if u not in user_preds:
                    user_preds[u] = []
                user_preds[u].append((i, p, l))

    # === Basic metrics ===
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    auc = roc_auc_score(all_labels, all_preds)
    rmse = np.sqrt(np.mean((all_labels - all_preds) ** 2))
    mae = np.mean(np.abs(all_labels - all_preds))

    print(f"\n--- Basic Metrics ---")
    print(f"AUC: {auc:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")

    # === Ranking metrics @K ===
    precisions, recalls, hits, maps = [], [], [], []

    for user, predictions in user_preds.items():
        predictions.sort(key=lambda x: x[1], reverse=True)
        total_relevant = sum([l for _, _, l in predictions])
        if total_relevant == 0:
            continue

        top_k = predictions[:K]
        top_k_labels = [l for _, _, l in top_k]

        # Precision, Recall, Hit
        precision = sum(top_k_labels) / K
        recall = sum(top_k_labels) / total_relevant
        hit = 1.0 if sum(top_k_labels) > 0 else 0.0

        # MAP@K
        ap_sum, correct = 0.0, 0
        for idx, (_, _, label) in enumerate(top_k, start=1):
            if label == 1:
                correct += 1
                ap_sum += correct / idx
        map_k = ap_sum / min(total_relevant, K)

        precisions.append(precision)
        recalls.append(recall)
        hits.append(hit)
        maps.append(map_k)

    # In kết quả trung bình
    print(f"\n--- Ranking Metrics @ {K} ---")
    print(f"Precision@{K}: {np.mean(precisions):.4f}")
    print(f"Recall@{K}:    {np.mean(recalls):.4f}")
    print(f"Hit@{K}:       {np.mean(hits):.4f}")
    print(f"MAP@{K}:       {np.mean(maps):.4f}")

# Gọi hàm đánh giá chính xác
evaluate_ranking_metrics(model, test_loader, device)

print("\n✅ ENCM Training & Evaluation Complete!")



Users: 42, Items: 139, Contexts: 27
Using device: cpu

=== Training ENCM ===


Epoch 1: 100%|██████████| 51/51 [00:00<00:00, 307.46it/s]

Epoch 1 | Train Loss: 0.6332



Epoch 2: 100%|██████████| 51/51 [00:00<00:00, 312.33it/s]


Epoch 2 | Train Loss: 0.5592


...

Epoch 8 | Train Loss: 0.4126


Epoch 9: 100%|██████████| 51/51 [00:00<00:00, 328.97it/s]


Epoch 9 | Train Loss: 0.3998


Epoch 10: 100%|██████████| 51/51 [00:00<00:00, 307.72it/s]

Epoch 10 | Train Loss: 0.3941

=== Evaluating ENCM ===

=== Evaluating ENCM ===

--- Basic Metrics ---
AUC: 0.7968
RMSE: 0.3882
MAE: 0.2812

--- Ranking Metrics @ 10 ---
Precision@10: 0.3448
Recall@10:    0.7876
Hit@10:       1.0000
MAP@10:       0.5520

✅ ENCM Training & Evaluation Complete!


In [15]:
# =====================================================
# 🎯 Context Similarity Post-filtering
# =====================================================
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

# Đường dẫn file
base_dir = './output_carskit_clean'
ratings_path = os.path.join(base_dir, 'ratings_with_context_id.csv')
context_matrix_path = os.path.join(base_dir, 'item_context_matrix.csv')

# 1️⃣ Đọc dữ liệu
ratings_df = pd.read_csv(ratings_path)
context_matrix = pd.read_csv(context_matrix_path)
print(f"📄 ratings: {ratings_df.shape}, context_matrix: {context_matrix.shape}")

# 2️⃣ Xác định context hiện tại (ví dụ context_id = 4)
CURRENT_CONTEXT = 4

# 3️⃣ Chuẩn bị vector context hiện tại
context_cols = [c for c in context_matrix.columns if c not in ['item', 'genre']]
current_context_vector = np.zeros(len(context_cols))
if str(CURRENT_CONTEXT) in context_cols:
    idx = context_cols.index(str(CURRENT_CONTEXT))
    current_context_vector[idx] = 1
else:
    raise ValueError(f"Context_id {CURRENT_CONTEXT} không có trong context_matrix")

# 4️⃣ Tính cosine similarity giữa context hiện tại và từng item
item_vectors = context_matrix[context_cols].values
similarity_scores = cosine_similarity(item_vectors, current_context_vector.reshape(1, -1)).flatten()

# 5️⃣ Gộp similarity vào ratings_df
sim_df = pd.DataFrame({'item': context_matrix['item'], 'context_similarity': similarity_scores})
ratings_df = ratings_df.merge(sim_df, on='item', how='left')

# 6️⃣ Tái xếp hạng
# Nếu muốn kết hợp điểm mô hình gốc (rating) với context_similarity:
ALPHA = 0.0  # 0.0 = chỉ dùng context similarity
ratings_df['final_score'] = ALPHA * ratings_df['label'] + (1 - ALPHA) * ratings_df['context_similarity']

# 7️⃣ Hàm đánh giá Precision@10 và MAP@10
def precision_at_k(y_true, y_score, k=10):
    top_k = np.argsort(y_score)[-k:][::-1]
    return np.mean(np.array(y_true)[top_k])

def map_at_k(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.array(y_true)[order][:k]
    cum_precisions = [np.mean(y_true[:i+1]) for i in range(len(y_true)) if y_true[i]]
    return np.mean(cum_precisions) if cum_precisions else 0.0

precisions, maps = [], []
for uid, group in ratings_df.groupby('user'):
    if group['label'].sum() == 0:
        continue
    precisions.append(precision_at_k(group['label'], group['final_score'], k=10))
    maps.append(map_at_k(group['label'], group['final_score'], k=10))

print(f"🎯 Precision@10: {np.mean(precisions):.4f}")
print(f"🎯 MAP@10: {np.mean(maps):.4f}")

# 8️⃣ Xuất file kết quả
output_path = os.path.join(base_dir, 'context_similarity_postfiltered.csv')
ratings_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Kết quả post-filtering đã lưu: {output_path}")


📄 ratings: (4012, 4), context_matrix: (139, 29)
🎯 Precision@10: 0.2776
🎯 MAP@10: 0.3981
✅ Kết quả post-filtering đã lưu: ./output_carskit_clean\context_similarity_postfiltered.csv
